In [1]:
"""
Name: Sashka Warner
Date: Dec 20, 2022
Desc:

Goal
It is your job to predict if a passenger survived the sinking of the Titanic or not.
For each in the test set, you must predict a 0 or 1 value for the variable.

Metric
Your score is the percentage of passengers you correctly predict.
"""

'\nName: Sashka Warner\nDate: Dec 20, 2022\nDesc:\n\nGoal\nIt is your job to predict if a passenger survived the sinking of the Titanic or not.\nFor each in the test set, you must predict a 0 or 1 value for the variable.\n\nMetric\nYour score is the percentage of passengers you correctly predict.\n'

In [21]:
import numpy as np
import pandas as pd
import re
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree, metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
# Load data
train_raw = pd.read_csv("./data/train.csv")
test_raw = pd.read_csv("./data/test.csv")
train = train_raw.copy()
test = test_raw.copy()
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# Check types and NA's
for i in train.columns:
    print("Column: ", i, ", Type: ", train[i].map(lambda x: type(x)).unique(), ", Has NAs: ", any(train[i].isna()))

Column:  PassengerId , Type:  [<class 'int'>] , Has NAs:  False
Column:  Survived , Type:  [<class 'int'>] , Has NAs:  False
Column:  Pclass , Type:  [<class 'int'>] , Has NAs:  False
Column:  Name , Type:  [<class 'str'>] , Has NAs:  False
Column:  Sex , Type:  [<class 'str'>] , Has NAs:  False
Column:  Age , Type:  [<class 'float'>] , Has NAs:  True
Column:  SibSp , Type:  [<class 'int'>] , Has NAs:  False
Column:  Parch , Type:  [<class 'int'>] , Has NAs:  False
Column:  Ticket , Type:  [<class 'str'>] , Has NAs:  False
Column:  Fare , Type:  [<class 'float'>] , Has NAs:  False
Column:  Cabin , Type:  [<class 'float'> <class 'str'>] , Has NAs:  True
Column:  Embarked , Type:  [<class 'str'> <class 'float'>] , Has NAs:  True


In [5]:
# Check types and NA's
for i in test.columns:
    print("Column: ", i, ", Type: ", test[i].map(lambda x: type(x)).unique(), ", Has NAs: ", any(test[i].isna()))

Column:  PassengerId , Type:  [<class 'int'>] , Has NAs:  False
Column:  Pclass , Type:  [<class 'int'>] , Has NAs:  False
Column:  Name , Type:  [<class 'str'>] , Has NAs:  False
Column:  Sex , Type:  [<class 'str'>] , Has NAs:  False
Column:  Age , Type:  [<class 'float'>] , Has NAs:  True
Column:  SibSp , Type:  [<class 'int'>] , Has NAs:  False
Column:  Parch , Type:  [<class 'int'>] , Has NAs:  False
Column:  Ticket , Type:  [<class 'str'>] , Has NAs:  False
Column:  Fare , Type:  [<class 'float'>] , Has NAs:  True
Column:  Cabin , Type:  [<class 'float'> <class 'str'>] , Has NAs:  True
Column:  Embarked , Type:  [<class 'str'>] , Has NAs:  False


In [6]:
# Check how many rows have NA for Age
print(train["Age"].isna().sum())
print(test["Age"].isna().sum())
print(test["Fare"].isna().sum())

# Replace categorical variable NAs with empty strings
# Replace numeric variable NAs with average values for the respective columns
train.fillna({
    "Cabin": "",
    "Embarked": "",
    "Age": train["Age"].mean(skipna=True),
}, inplace=True)

test.fillna({
    "Cabin": "",
    "Age": train["Age"].mean(skipna=True),
    "Fare": train["Fare"].mean(skipna=True),
}, inplace=True)

print(train["Age"].isna().sum())
print(test["Age"].isna().sum())
print(test["Fare"].isna().sum())

177
86
1
0
0
0


In [7]:
# Check types and NA's
for i in train.columns:
    print("Column: ", i, ", Type: ", train[i].map(lambda x: type(x)).unique(), ", Has NAs: ", any(train[i].isna()))

Column:  PassengerId , Type:  [<class 'int'>] , Has NAs:  False
Column:  Survived , Type:  [<class 'int'>] , Has NAs:  False
Column:  Pclass , Type:  [<class 'int'>] , Has NAs:  False
Column:  Name , Type:  [<class 'str'>] , Has NAs:  False
Column:  Sex , Type:  [<class 'str'>] , Has NAs:  False
Column:  Age , Type:  [<class 'float'>] , Has NAs:  False
Column:  SibSp , Type:  [<class 'int'>] , Has NAs:  False
Column:  Parch , Type:  [<class 'int'>] , Has NAs:  False
Column:  Ticket , Type:  [<class 'str'>] , Has NAs:  False
Column:  Fare , Type:  [<class 'float'>] , Has NAs:  False
Column:  Cabin , Type:  [<class 'str'>] , Has NAs:  False
Column:  Embarked , Type:  [<class 'str'>] , Has NAs:  False


In [8]:
# Check types and NA's
for i in test.columns:
    print("Column: ", i, ", Type: ", test[i].map(lambda x: type(x)).unique(), ", Has NAs: ", any(test[i].isna()))

Column:  PassengerId , Type:  [<class 'int'>] , Has NAs:  False
Column:  Pclass , Type:  [<class 'int'>] , Has NAs:  False
Column:  Name , Type:  [<class 'str'>] , Has NAs:  False
Column:  Sex , Type:  [<class 'str'>] , Has NAs:  False
Column:  Age , Type:  [<class 'float'>] , Has NAs:  False
Column:  SibSp , Type:  [<class 'int'>] , Has NAs:  False
Column:  Parch , Type:  [<class 'int'>] , Has NAs:  False
Column:  Ticket , Type:  [<class 'str'>] , Has NAs:  False
Column:  Fare , Type:  [<class 'float'>] , Has NAs:  False
Column:  Cabin , Type:  [<class 'str'>] , Has NAs:  False
Column:  Embarked , Type:  [<class 'str'>] , Has NAs:  False


In [9]:
# Examine cabin data
train["Cabin"].unique()

array(['', 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6', 'C23 C25 C27',
       'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33', 'F G73', 'E31',
       'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101', 'F E69', 'D47',
       'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4', 'A32', 'B4',
       'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35', 'C87', 'B77',
       'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19', 'B49', 'D',
       'C22 C26', 'C106', 'C65', 'E36', 'C54', 'B57 B59 B63 B66', 'C7',
       'E34', 'C32', 'B18', 'C124', 'C91', 'E40', 'T', 'C128', 'D37',
       'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44', 'A34', 'C104',
       'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14', 'B37', 'C30',
       'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38', 'B39', 'B22',
       'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68', 'B41', 'A20',
       'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48', 'E58', 'C126',
       'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63', 'C62 C64',
       

In [10]:
# Extract deck from cabin values
train["Floor"] = train["Cabin"].map(lambda x: re.sub("[^a-zA-Z]", "", x))
test["Floor"] = test["Cabin"].map(lambda x: re.sub("[^a-zA-Z]", "", x))
train.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Floor
886,887,0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.00,,S,
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.00,B42,S,B
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.699118,1,2,W./C. 6607,23.45,,S,
889,890,1,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.00,C148,C,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.000000,0,0,370376,7.75,,Q,


In [11]:
#Get features of interest
train_sel = train[["Pclass", "Sex", "Floor", "Embarked"]]
test_sel = test[["Pclass", "Sex", "Floor", "Embarked"]]
print(train_sel.shape)
print(test_sel.shape)

(891, 4)
(418, 4)


In [12]:
# Convert integer data to one hot encoding
train_encoder = OneHotEncoder(sparse=False)
train_encoded = train_encoder.fit_transform(train_sel)
train_encoded = pd.DataFrame(train_encoded, columns=train_encoder.get_feature_names_out(), dtype=int)
train_encoded.reset_index(drop=True, inplace=True)

# Do the same for the test data
test_encoder = OneHotEncoder(sparse=False)
test_encoded = test_encoder.fit_transform(test_sel)
test_encoded = pd.DataFrame(test_encoded, columns=test_encoder.get_feature_names_out(), dtype=int)
test_encoded.reset_index(drop=True, inplace=True)

# Examine output
print(train_encoded.columns)
print(test_encoded.columns)

Index(['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Floor_',
       'Floor_A', 'Floor_B', 'Floor_BB', 'Floor_BBB', 'Floor_BBBB', 'Floor_C',
       'Floor_CC', 'Floor_CCC', 'Floor_D', 'Floor_DD', 'Floor_E', 'Floor_F',
       'Floor_FE', 'Floor_FG', 'Floor_G', 'Floor_T', 'Embarked_', 'Embarked_C',
       'Embarked_Q', 'Embarked_S'],
      dtype='object')
Index(['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Floor_',
       'Floor_A', 'Floor_B', 'Floor_BB', 'Floor_BBB', 'Floor_BBBB', 'Floor_C',
       'Floor_CC', 'Floor_CCC', 'Floor_D', 'Floor_DD', 'Floor_E', 'Floor_EE',
       'Floor_F', 'Floor_FE', 'Floor_FG', 'Floor_G', 'Embarked_C',
       'Embarked_Q', 'Embarked_S'],
      dtype='object')


In [13]:
# Add extra column to training data for "Floor_EE" value
train_floor_EE = np.zeros(len(train_encoded))
train_encoded["Floor_EE"] = train_floor_EE
train_encoded = train_encoded[['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Floor_',
       'Floor_A', 'Floor_B', 'Floor_BB', 'Floor_BBB', 'Floor_BBBB', 'Floor_C',
       'Floor_CC', 'Floor_CCC', 'Floor_D', 'Floor_DD', 'Floor_E', 'Floor_EE',
       'Floor_F', 'Floor_FE', 'Floor_FG', 'Floor_G', 'Embarked_C',
       'Embarked_Q', 'Embarked_S']]

In [14]:
# Add encoded data back to main train df
train_numeric = train[["Age", "SibSp", "Parch", "Fare"]]
train_numeric.reset_index(drop=True, inplace=True)

# Add encoded data back to main test df
test_numeric = test[["Age", "SibSp", "Parch", "Fare"]]
test_numeric.reset_index(drop=True, inplace=True)

In [15]:
# Concatenate data frames
X_train = pd.concat([train_numeric, train_encoded], axis=1)
#print(X_train.head())
print(X_train.shape)

X_test = pd.concat([test_numeric, test_encoded], axis=1)
#print(X_test.head())
print(X_test.shape)

(891, 29)
(418, 29)


In [16]:
# Extract targets
y_train = train["Survived"]
print(y_train.head())
y_train.shape

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64


(891,)

In [22]:
# Create Decision tree classifier
#clf = DecisionTreeClassifier()

# Create Random forest classifier
#clf = RandomForestClassifier()

# Create Gradient Boost classifier
clf = GradientBoostingClassifier(n_estimators=50, learning_rate=1.0, max_depth=1, random_state=0)

# Train the classifier
clf = clf.fit(X_train, y_train)

In [23]:
# Predict the response for the test dataset
y_pred = clf.predict(X_test)
y_pred.shape

(418,)

In [24]:
# Format data for submission
y_pred_df = pd.concat([test["PassengerId"], pd.DataFrame(y_pred, columns = ["Survived"])], axis=1)
y_pred_df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [20]:
# Save to csv
y_pred_df.to_csv("./predictions/submission.csv", index=False)